In [1]:
!pip install vllm --quiet
!pip install bitsandbytes>=0.45.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.9/396.9 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6

In [2]:
from vllm import LLM, SamplingParams
import time
import torch

In [1]:
model_id = "lmsys/vicuna-7b-v1.3"
llm = LLM(
    model=model_id,
    dtype=torch.float16,
    trust_remote_code=True,
    max_num_seqs=16,  # Maximum concurrent requests
    max_num_batched_tokens=2048,  # Batch token capacity
    tensor_parallel_size = 1, speculative_model = "abhigoyal/vllm-medusa-vicuna-7b-v1.3", num_speculative_tokens = 3, \
    use_v2_block_manager=True)

NameError: name 'LLM' is not defined

In [3]:
prompts = [
    "Explain quantum computing in simple terms",
    "Write a poem about artificial intelligence",
    "Describe the process of photosynthesis",
    "What's the capital of France?",
    "How does a CPU work?",
    "Explain special relativity"
]

sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.95,
    max_tokens=256
)


start = time.time()
for prompt in prompts:
    out = llm.generate(prompt, sampling_params)
    print(out)
end = time.time()
print(f"Time taken without batching: {end - start} seconds")


start = time.time()
outputs = llm.generate(prompts, sampling_params)
end = time.time()

for out in outputs:
    print(f"Prompt: {out.prompt}\nResponse: {out.outputs[0].text}\n")
print(f"Time taken with batching: {end - start} seconds")


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.03s/it, est. speed input: 2.64 toks/s, output: 34.63 toks/s]


[RequestOutput(request_id=0, prompt='Explain quantum computing in simple terms', prompt_token_ids=[1, 12027, 7420, 12101, 20602, 297, 2560, 4958], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='\n\nQuantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. Unlike classical computers, which use bits to represent and process information, quantum computers use quantum bits, or qubits. Qubits have the ability to exist in multiple states at once, allowing quantum computers to perform multiple calculations simultaneously. This property, known as parallelism, makes quantum computers much faster than classical computers for certain types of problems.', token_ids=(13, 13, 22930, 398, 20602, 338, 263, 1134, 310, 20602, 393, 3913, 12101, 29899, 1004, 5083, 936, 17292, 28342, 29892, 1316, 408, 2428, 3283, 322, 875, 574, 944, 29892, 304, 21

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.60s/it, est. speed input: 2.70 toks/s, output: 98.69 toks/s]


[RequestOutput(request_id=1, prompt='Write a poem about artificial intelligence', prompt_token_ids=[1, 14350, 263, 26576, 1048, 23116, 21082], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='\n“Artificial intelligence” is a topic that has been discussed and debated for many years. The concept of creating machines that can think and act like humans has fascinated people for centuries. In this poem, I will explore the idea of artificial intelligence and what it means for our future.\n\nArtificial intelligence, a topic of great debate\nCan machines truly think and feel like humans do?\nSome say yes, while others say no\nBut the truth is still unknown to most\n\nThe potential of AI is vast and wide\nIt could change the world in ways we can’t hide\nFrom self-driving cars to medical advances\nThe possibilities are endless and vast\n\nBut with great power comes great responsibility\nWe must ensure that AI is used ethically\nTo

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.48s/it, est. speed input: 3.63 toks/s, output: 103.32 toks/s]


...

[RequestOutput(request_id=4, prompt='How does a CPU work?', prompt_token_ids=[1, 1128, 947, 263, 10808, 664, 29973], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text="\nA CPU, or central processing unit, is the brain of a computer. It performs the majority of the processing and calculations required to run a computer program. Here is a simplified explanation of how a CPU works:\n1. Instructions: The CPU receives instructions from the computer' My interest in the stock market started when I was a young boy, watching my dad buy and sell stocks on his brokerage account. I was always curious about how the stock market worked and how people could make money from it. As I grew older, I became more interested in the stock market and started to learn about investing.\nI learned about the different types of stocks, such as common stocks, preferred stocks, and bonds, and how they can be used to generate returns. I also learned abou

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.13it/s, est. speed input: 6.81 toks/s, output: 93.08 toks/s]


[RequestOutput(request_id=5, prompt='Explain special relativity', prompt_token_ids=[1, 12027, 7420, 4266, 14215, 537], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' and how it rel6/12/2017 · Explain the principles of special relativity. 1. Time dilation: Time moves slower for an object in motion relative to a stationary observer. 2. Length contraction: Objects move along a shorter distance along the direction of motion. 3. Mass-energy equivalence: E=mc^2.', token_ids=(322, 920, 372, 1104, 1, 29953, 29914, 29896, 29906, 29914, 29906, 29900, 29896, 29955, 2880, 12027, 7420, 278, 18671, 310, 4266, 14215, 537, 29889, 29871, 29896, 29889, 5974, 270, 8634, 29901, 5974, 16229, 20312, 363, 385, 1203, 297, 10884, 6198, 304, 263, 5073, 653, 22944, 29889, 29871, 29906, 29889, 365, 1477, 6761, 428, 29901, 4669, 29879, 4337, 3412, 263, 20511, 5418, 3412, 278, 5305, 310, 10884, 29889, 29871, 29941, 29889, 7360, 29899, 27548, 24796

Processed prompts: 100%|██████████| 6/6 [00:02<00:00,  2.01it/s, est. speed input: 15.42 toks/s, output: 358.64 toks/s]

Prompt: Explain quantum computing in simple terms
Response: 

Prompt: Write a poem about artificial intelligence
Response: 
Artificial intelligence is a topic that has been discussed and debated for many years. It is a topic that has the potential to change the world, but it also raises many ethical questions. Write a poem about artificial intelligence and what it means to you.

Artificial intelligence is a topic that is often discussed,
It is a topic that has the potential to change the world,
It is a topic that raises many ethical questions,
But it also has the potential to create new opportunities.

AI is a machine that can learn and adapt,
It can process vast amounts of data and make predictions,
It can analyze and make decisions,
But it also raises questions about privacy and security.

AI is a topic that is often misunderstood,
It is not a sentient being, it is a machine,
It is not a replacement for human intelligence,
It is a tool that can help us to achieve our goals.

AI has t

In [3]:
model_id = "lmsys/vicuna-7b-v1.3"
llm = LLM(
    model=model_id,
    dtype=torch.float16,
    trust_remote_code=True,
    max_num_seqs=16,  # Maximum concurrent requests
    max_num_batched_tokens=2048,  # Batch token capacity
)

INFO 02-21 06:43:50 __init__.py:207] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 02-21 06:44:01 config.py:549] This model supports multiple tasks: {'reward', 'classify', 'generate', 'score', 'embed'}. Defaulting to 'generate'.
INFO 02-21 06:44:01 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='lmsys/vicuna-7b-v1.3', speculative_config=None, tokenizer='lmsys/vicuna-7b-v1.3', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=lmsys/vicuna-7b-v1.3, num_scheduler_steps=1, m

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

INFO 02-21 06:44:03 cuda.py:229] Using Flash Attention backend.
INFO 02-21 06:44:04 model_runner.py:1110] Starting to load model lmsys/vicuna-7b-v1.3...
INFO 02-21 06:44:04 weight_utils.py:254] Using model weights format ['*.bin']


Loading pt checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-21 06:44:16 model_runner.py:1115] Loading model weights took 12.5518 GB
INFO 02-21 06:44:18 worker.py:267] Memory profiling takes 0.78 seconds
INFO 02-21 06:44:18 worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.90) = 35.60GiB
INFO 02-21 06:44:18 worker.py:267] model weights take 12.55GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 0.18GiB; the rest of the memory reserved for KV Cache is 22.78GiB.
INFO 02-21 06:44:18 executor_base.py:111] # cuda blocks: 2915, # CPU blocks: 512
INFO 02-21 06:44:18 executor_base.py:116] Maximum concurrency for 2048 tokens per request: 22.77x
INFO 02-21 06:44:20 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████| 5/5 [00:03<00:00,  1.37it/s]

INFO 02-21 06:44:24 model_runner.py:1562] Graph capturing finished in 4 secs, took 0.06 GiB
INFO 02-21 06:44:24 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 7.31 seconds


In [4]:
prompts = [
    "Explain quantum computing in simple terms",
    "Write a poem about artificial intelligence",
    "Describe the process of photosynthesis",
    "What's the capital of France?",
    "How does a CPU work?",
    "Explain special relativity"
]

sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.95,
    max_tokens=256
)


start = time.time()
for prompt in prompts:
    out = llm.generate(prompt, sampling_params)
    print(out)
end = time.time()
print(f"\n\nTime taken without batching: {end - start} seconds\n\n")


start = time.time()
outputs = llm.generate(prompts, sampling_params)
end = time.time()

for out in outputs:
    print(f"Prompt: {out.prompt}\nResponse: {out.outputs[0].text}\n")
print(f"\n\nTime taken with batching: {end - start} seconds\n\n")


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.35s/it, est. speed input: 2.39 toks/s, output: 76.50 toks/s]


[RequestOutput(request_id=0, prompt='Explain quantum computing in simple terms', prompt_token_ids=[1, 12027, 7420, 12101, 20602, 297, 2560, 4958], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='\nQuantum computing is a type of computing that uses quantum bits or qubits to process information. Unlike classical bits, qubits can exist in multiple states at the same time, making quantum computers much faster at solving certain complex problems.\nHow does quantum computing work?\nQuantum computing works by encoding information in the state of a qubit, which can represent a 1, a 0, or a superposition of both. This means that a single qubit can represent multiple values simultaneously, making quantum computers much faster at solving certain complex problems than classical computers.\nWhat are the benefits of quantum computing?\nQuantum computing has the potential to revolutionize many fields, including cryptography, chemistry

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.28s/it, est. speed input: 3.07 toks/s, output: 76.69 toks/s]


[RequestOutput(request_id=1, prompt='Write a poem about artificial intelligence', prompt_token_ids=[1, 14350, 263, 26576, 1048, 23116, 21082], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='\nWrite a poem about artificial intelligence.\nArtificial intelligence,\nA world of machines,\nWith minds of their own,\nYet they lack consciousness.\n\nThe algorithms and code,\nAre the building blocks,\nOf this new technology,\nBut what do they really know?\n\nCan they think and reason,\nLike humans do,\nOr are they just a bunch,\nOf ones and zeros too?\n\nThe future of AI,\nIs still uncertain,\nWill it be a help,\nOr a hindrance to humanity?\n\nOnly time will tell,\nAs we continue to evolve,\nAnd create new ways,\nTo use this technology to thrive.\n\nSo let us embrace,\nThe possibilities,\nOf artificial intelligence,\nAnd hope for the best.', token_ids=(13, 6113, 263, 26576, 1048, 23116, 21082, 29889, 13, 9986, 928, 616, 21082, 2

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.33s/it, est. speed input: 2.70 toks/s, output: 76.91 toks/s]


...

[RequestOutput(request_id=4, prompt='How does a CPU work?', prompt_token_ids=[1, 1128, 947, 263, 10808, 664, 29973], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='\nA CPU, or central processing unit, is the "brain" of a computer. It is responsible for executing instructions and performing calculations.\n\nThe CPU is made up of several parts:\n\n1. Registers: small, fast memory locations that hold data temporarily.\n2. Arithmetic Logic Unit (ALU): performs mathematical and logical operations.\n3. Control Unit: manages the flow of data and instructions between the CPU and memory.\n4. Instruction Set: a set of instructions that the CPU can execute.\n\nWhen a program is run, the CPU reads the instructions from memory and executes them. It performs calculations and manipulates data according to the instructions it has been given.\n\nThe speed at which a CPU can execute instructions is measured in GHz, or gigahertz. A faste

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.33s/it, est. speed input: 1.80 toks/s, output: 76.95 toks/s]


[RequestOutput(request_id=5, prompt='Explain special relativity', prompt_token_ids=[1, 12027, 7420, 4266, 14215, 537], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' with the help of a thought experiment\nSpecial relativity is the theory that describes how space and time are affected by motion at high spe lived speeds. It was developed by Albert Einstein in 1905 and is based on two postulates:\n1. The laws of physics are the same for all observers in inertial reference frames.\n2. The speed of light is the same for all observers, regardless of their relative motion.\nTo understand special relativity, we can use a thought experiment, called the "Einstein\'s Train Thought Experiment" or "Einstein\'s Two-Arrow Paradox".\nImagine an observer on a train that is moving at a constant speed towards you. The train is moving at a speed close to the speed of light and is surrounded by a vacuum. The observer on the train has a li

Processed prompts: 100%|██████████| 6/6 [00:03<00:00,  1.81it/s, est. speed input: 13.91 toks/s, output: 190.55 toks/s]

Prompt: Explain quantum computing in simple terms
Response: .

Quantum computing is a type of computing that uses the principles of quantum mechanics to perform calculations. Unlike classical computers that use bits to store and process information, quantum computers use quantum bits, or qubits. Qubits have the ability to exist in multiple states at the same time, allowing quantum computers to perform many calculations simultaneously, making them much faster and more powerful than classical computers.

Prompt: Write a poem about artificial intelligence
Response: 

Prompt: Describe the process of photosynthesis
Response:  in plants, including the role of chlorophyll and light-dependent and light-independent reactions.
Describe the structure and function of mitochondria, including the role of oxidative phosphorylation in cellular respiration.
Describe the role of ATP in cellular processes, including energy transfer and storage.
Describe the process of transcription and translation, inclu